In [46]:
import keras
import gradio as gr
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.regularizers import l2


In [47]:
#  to split the data of training and testing sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [48]:
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# conversion of class vectors to matrices of  binary class 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [50]:
batch_size = 128
num_classes = 10
epochs = 20  # Increased number of epochs for better training
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))  # Added an additional convolutional layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, kernel_regularizer=l2(0.001), activation='relu'))  # Added L2 regularization
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),metrics=['accuracy'])  # Changed optimizer to Adam


In [51]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_6 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 12, 12, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                  

In [52]:
history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")
model.save('mnist.h5')
print("Saving the bot as mnist.h5")

Epoch 1/20
469/469 [==============================] - 103s 219ms/step - loss: 0.4213 - accuracy: 0.9302 - val_loss: 0.1651 - val_accuracy: 0.9853
Epoch 2/20
469/469 [==============================] - 100s 213ms/step - loss: 0.1781 - accuracy: 0.9763 - val_loss: 0.1228 - val_accuracy: 0.9897
Epoch 3/20
469/469 [==============================] - 93s 199ms/step - loss: 0.1517 - accuracy: 0.9797 - val_loss: 0.1140 - val_accuracy: 0.9894
Epoch 4/20
469/469 [==============================] - 98s 209ms/step - loss: 0.1411 - accuracy: 0.9821 - val_loss: 0.1095 - val_accuracy: 0.9905
Epoch 5/20
469/469 [==============================] - 89s 190ms/step - loss: 0.1342 - accuracy: 0.9829 - val_loss: 0.1115 - val_accuracy: 0.9889
Epoch 6/20
469/469 [==============================] - 90s 192ms/step - loss: 0.1294 - accuracy: 0.9840 - val_loss: 0.0997 - val_accuracy: 0.9927
Epoch 7/20
469/469 [==============================] - 85s 180ms/step - loss: 0.1268 - accuracy: 0.9845 - val_loss: 0.1011 - val_

In [53]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.09064210951328278
Test accuracy: 0.9940000176429749


In [56]:
import gradio as gr
import numpy as np
import tensorflow as tf

# Load the trained MNIST model
model = tf.keras.models.load_model('mnist.h5')

# Define a function to preprocess the input sketchpad image
def preprocess_image(image):
    
    image = Image.fromarray(np.uint8(image))
    # Convert the image to grayscale
    image = image.convert('L')
    # Resize the image to 28x28 pixels (MNIST image size)
    image = image.resize((28, 28))
    # Convert the image to a NumPy array
    image = np.array(image)
    # Normalize the image pixel values
    image = image / 255.0
    # Reshape the image to match the model's input shape
    image = image.reshape(1, 28, 28, 1)
    return image

# Define a function to make predictions on the preprocessed image
def predict_image(image):
    # Make a prediction using the model
    predictions = model.predict(image)
    # Get the predicted digit and its probability
    digit = np.argmax(predictions)
    probability = predictions[0][digit]
    return digit, probability

# Define the Gradio interface
def sketchpad_digit_recognition(image):
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    # Make predictions on the preprocessed image
    digit, probability = predict_image(preprocessed_image)
    # Return the predicted digit and its probability
    return digit, probability

# Set up the Gradio interface
# sketchpad = gr.inputs.Sketchpad()
interface = gr.Interface(fn=sketchpad_digit_recognition, inputs="sketchpad",outputs=output_text, title='Sketchpad Digit Recognition')
interface.launch()

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 32ms/step
